Multiplicity of Search Trees:
 - Recall,for a node x in search tree, every key in left child < x, every key in right child > x. 
 - For a given set of keys, there are lots of valid search trees. 
 - "Best" searh tree for a set of keys is a balaned tree (like a red-black tree). 
     - Worst case search time is O(height) = O(logn)

Non-Uniformity:
 - Suppose have keys x(0.8 prob searched for), y (0.1 prob searched for), z (0.1 prob searched for)
 - In this case, perfectly balanced tree is still good but more important to consider height of most searched-for objects. 
     - Worst case searh time still based on O(height), but may improve search time on average. 

Problem:
 - Input: Frequencies p1, p2, ....pn for items 1 to n (Assume items in sorted order where 1 < 2 <3 ... < n)
     - Applications exist where may learn how often certain items are searched for
 - Goal: Compute a valid search tree that minimizes the average/weighted search time.
     - Search Time Avg = C(T) = sum(pi * (depth of i + 1)). Assuming for now that all searches are successful so only thing that ever gets searched is in the tree. 
         - Note, depth of root = 0. 
         - Allow for pi to be arbitary positive numbers. 
 - Example: If T is a red-black tree, then C(T) = O(logn). 
 
Comparison with Huffman Codes:
 - Similarities:
     - Outputs a binary tree
     - Goal is (essentially) to minimize average depth with respect to given probabilities
 - Differences:
     - In Huffman, output a binary code and the contraint was based on them being prefix-free (i.e. symbols are only at leaves)
     - Here, no prefix-free constraint so lables will be at each node of tree. But, must also deal with the search tree property bc items have both frequency and ordering. 

## Optimal Structure

Why Greedy's are insufficient:
 - Intuition: objects with highest frequency to be at or near the root, others to be near the leaves
 - Ideas for Greedy:
     - Bottom-up approach: populate lowest level with least frequently accessed keys. 
         - Sometimes, object with lowest frequency does not belong on lowest level though. Due to binary tree invariant. greedy not enough. 
     - Other greedy's do not necessarily work (such as top-down approach, putting most frequent at root). 

Choosing the Root:
 - Problem: With top-down approach, the choice of root has hard-to-predict repurcussions further down the tree (for both greedy and divide/conquer approaches).
     - Root value in consideration alone is not good bc larger effects on rest of tree
 - But, if knew what the root was, could optimally recurse on smaller subproblems. So, what do we know?
 
What do we know about the root?:
 - i.e. maybe can try all possibilities within a dynamic programming algorithm
 - Suppose an optimal binary search tree for keys 1 to n has root r, left subtree T1, right subtree T2. Must be true that T1 is optimal for keys (1 to r-1) and T2 optimal for keys (r + 1 to n), the exact items that will be in each tree respectively. 

### Proof

Let T be an optimal BST for keys 1 to n with frequencies p1 to pn. Suppose T has root r.
 - Suppose for contradiction that T1 is not optimal for (1, 2, ... r-1) (similcar case with T2) with C(T1*) < C(T1); T1* is some better subtree T1 for those keys. 
 - Obtain T* from T by "cutting and pasting" T1* in for T1. Basically, replace T1 with T1*
     - To complete contradiction and proof, only need to show that C(T*) < C(T)
 - A Calculation: 
     - C(T) = sum(pi * (depth of i + 1 aka search time for i in T)), express i terms of search times in T1 and T2. 
         - C(T) = pr * 1 (root time) + sum(pi * search time for i in T, i from 1 to r-1) + sum(pi * search time for i in T, i from r + 1 to n)
         - Note that search time for any i in T1 or T2, search time = 1 + search time for i in T1.
         - C(T) = sum(pi, all n), const independent of T, + [C(T1) = sum(pi * search for i in T1, 1 to r-1)] + [C(T2) = sum(pi * search for i in T2, r + 1 to n)]
         - This is true for any T
 - Similarly: C(T*) = sum(pi, all n) + C(T1*) + C(T2)
 - So, by above, we know that for some T, T1 and T2 must be optimal for the keys they possess else there will exist a better T* with T1* and T2*. Done.

## DP Algorithm

Here, we care about subproblems corresponding to subtrees. Items in a subproblem are either a prefix or suffix of t he original problem. Subproblem obtained by throwing away a prefix or throwing away a suffix from what we begin with. 

Let there be n original itmes. of subsets S (of 1 to n), might need to compute the optimal BST for contiguous itervals {S = (i, i+1,...,j) for every i <= j}
 - i is smallest item in subproblem, j is the biggest
 - EX: Have 100 items. At some point, will be trying out root # 23 to see how it does
     - Recurse once on items 1 through 22 (T1) and items 24 to 100 (T2).
     - Consider T1. In T1, will be trying all possible roots here (like root 17). 2 recursive calls, one on 1 to 16, second on 18 to 22. This second is a suffix of the original 1 to 22. 
     - At every step, either deleting prefix or suffix. So, not true always will have prefix or suffix of orignal set of items.
     - True though that will always have some contiguous set of items. 

Recurrence:
 - Notation: for 1 <= i <= j <= n, let Cij = weighted search cost of an optimal BST for the items i to j. 
 - for every 1 <= i <= j <= n:
    - Let r be root, somewhere between i and j inclusive
    - Cij = min(
         - sum(pk, k in i to j) + Ci,r-1 + Cr+1,j) for r in i to j inclusive
         - Formula from optimal solution; interpret Cxy = 0 if x > y
         
Correctness: Optimal structure narrows candidates down to (j-1 +1) possible roots (which defines optimal struct), recurrence picks the best by brute force

Why done:
 - Consider n = 6:
     - First set considered is (1, 2)
         - This set will be used to calculate whether root 3 is optimal or not, which feeds upwards for more. 
     - (2,3), considers if root 1 optimal (but not as final root, just a subtree or something like that) 

Important: Solve smallest subproblems with fewest number (j - i + 1) of items first. 

Let A = 2-D array
 - for s = 0 to (n - 1) #controls subproblem size, represents j - i
     - for i = 1 to n (so i + s is j)
         - A[i, i + s] = for r = i to i + s, min(:
             - sum(pk, i to i + s) + A[i, r-1] + A[r + 1, i + s] (Note, in real implementation, if r is i or i + s, interpret those lookups as 0 for 1st and 2nd respectively. Also make sure lookups above are constant. Since, regardless of what choice of root is, the two relevant subproblems are going to involve strictly fewer items than what started with, they will definitely be computed already
 - Return A[1, n]

Understanding 2-D array:
 - j is y axis, i is x axis. 
 - for diagonal, i = j. The bottom-right of this is empty bc only bother with j > i. 
 - First iteration of double for loop, algo solves in turn each of the n single problems. A[1,1], A[2,2]. A[i,i] = probability of item i. 
     - Then, each increment of s, goes up to top-left one diagonal at a time. Think of it like a graph
         - Each one goes up: j = i + 1, j = i + 2, etc. Moves upwards. 
         
Running Time:
 - Thinking of array, fills in 1/2 of the n x n array. Effectively n^2/2 or O(n^2) subproblems
 - O(j - 1) time to compute A[i, j] because tries every possble root. Each item in subproblem is a candidate subproblem 
 - Total, Theta(n^3). Jesus fuck but still far better than brute force. n in 1000's will have you screwed p much. 
 - But, can speed up a lot. Optimal version correctly fills entire tabl in only Theta(n^2) time [Theta(1) on average per subproblem]
     - Idea: piggyback on work done in previous subproblems to avoid trying all possible combinations. Can use previous ones to make inferences abt subsets or smthn. Can apply to 10000s.